# Stacked Denoising Autoencoder regression

Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, GaussianNoise, LeakyReLU
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

C:\Users\faizm\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Read and load datasets

In [3]:
# load dataset
dataframe = pandas.read_csv("flesh_color.txt", delim_whitespace=True, header=None)
dataset = dataframe.values
print("data shape :", dataframe.shape)
data_len = dataset.shape[1]
print("no. of features : ", data_len )

data shape : (86, 15441)
no. of features :  15441


Splitting the dataset into taining and testing set

In [4]:
# split into input (X) and output (Y) variables
train, test = train_test_split(dataset, test_size=0.2)
trainX = train[:,0:data_len-1]
trainY = train[:,data_len-1]
testX = test[:,0:data_len-1]
testY = test[:,data_len-1]

print("trainX : ", trainX.shape)
print("trainY : ", trainY.shape)
print("testX : ", testX.shape)
print("testY : ", testY.shape)
#np.savetxt('Y.txt', trainY)
#np.savetxt('Y_test.txt', testY)

trainX :  (68, 15440)
trainY :  (68,)
testX :  (18, 15440)
testY :  (18,)


Normalizing the data

In [5]:
# Normalizing train data

max_abs_scaler = MaxAbsScaler()
normX = max_abs_scaler.fit_transform(trainX)
input_shape = normX.shape
#print("X norm : ", normX[:,15439])
print("normX shape : ", input_shape)
#np.savetxt('xnorm.txt', normX)
small_train = normX[:, 0:8000]

# Normalizing test data

max_abs_scaler = MaxAbsScaler()
normXt = max_abs_scaler.fit_transform(testX)
#np.savetxt('test_set.txt', normXt)
print("X norm shape :", normXt.shape)
small_test = normXt[:, 0:8000]

print("small train : ", small_train.shape)
print("smnall test : ", small_test.shape)

normX shape :  (68, 15440)
X norm shape : (18, 15440)
small train :  (68, 8000)
smnall test :  (18, 8000)


In [6]:
earlyStopping = EarlyStopping(monitor='loss', patience=50)
K.set_learning_phase(1)
checkpointer = ModelCheckpoint(filepath='flesh_color.h5', verbose=1, save_best_only=True)
input_dims = data_len-1

# 1 - Simple Denoising Autoencoder

COMPULSORY!!! run the function below before go to the Stacked Denoising Autoencoder

In [7]:
def dae (inputX, input_dims, output_dims, epoch, activation, loss, opti):
    
    #input_dims = inputX.shape
    #print("input dims : ", input_dims)
    #config = tf.ConfigProto()
    #config.gpu_options.per_process_gpu_memory_fraction = 1.0
    #session = tf.Session(config=config)
    
    model = Sequential()

    model.add(Dense(input_dims, input_dim = input_dims))
    model.add(GaussianNoise(0.5))
    model.add(Dense(output_dims, activation= activation, kernel_regularizer = regularizers.l1(0.01)))
    model.add(Dense(input_dims, activation= activation))
    
    model.compile(loss = loss, optimizer = opti)
    model.fit(inputX, inputX, epochs = epoch, batch_size = 4)
    model.summary()
    
    return model
    

Don't need to run code below if you want to go straight to the Stacked Denoising Autoencoder. This code is to show how the Denoising Autoencoder works.

In [ ]:
autoencoder = dae(normX, input_dims =15440, output_dims = 10000, epoch = 3, activation = 'relu', loss = 'mse', opti = 'adam')

Epoch 1/3


In [ ]:
autoencoder.summary()

# 2 - Stacked Denoising Autoencoder - pretrain

In [ ]:
# Hyper-parameter

layers = [input_dims, 10000, 5000, 3000, 1000, 500, 300, 100]
epoch = 3
optimizer = 'adamax'
activation = 'relu'
loss = 'mse'

In [ ]:
def sdae_pretrain (inputX, layers, activation, epoch, optimizer, loss):
    
    encoder = []
    decoder = []
    ae = []
    
    for i in range(len(layers)-1):
            # Greedily train each layer
            print("Now pretraining layer {} [{}-->{}]".format(i+1, layers[i], layers[i+1]))

            input_dims = layers[i]
            output_dims = layers[i+1]
            
            autoencoder = dae(inputX, input_dims, output_dims, epoch, activation, loss, optimizer)
            
            enc = Sequential()
            enc.add(Dense(output_dims, input_dim=input_dims))
            enc.compile(loss=loss, optimizer=optimizer)
            enc.layers[0].set_weights(autoencoder.layers[2].get_weights())
            inputX = enc.predict(inputX)
            print("check dimension : ", inputX.shape)
            enc.summary()
            #print(i)
            encoder.append(autoencoder.layers[2].get_weights())
            decoder.append(autoencoder.layers[3].get_weights())
            ae.append(autoencoder)

    
    return ae

In [ ]:
test = sdae_pretrain(normX, layers = layers, activation = activation, epoch = epoch, optimizer = optimizer, loss = loss)

OPTIONAL: Saving the models for each layer

In [ ]:
for i in range(len(test)):
    
    test[i].save("pre_train_l1" + str(i) + ".hd5")

# 3 - Stacked Denoising Autoencoder - fine tuning

In [ ]:
test = []

for i in range (len(layers)-1):
    
    test.append(load_model("pre_train_l1"+ str(i) + ".hd5"))

In [ ]:
len(test)

In [ ]:
def fine_tuning(weights, inputX, inputXt, inputY, inputYt, layers, epoch, activation, batch, optimizer, loss):

    encoder = []
    decoder = []

    for i in range(len(test)):
    
        encoder.append(test[i].layers[2].get_weights())
        decoder.append(test[i].layers[3].get_weights())
    
    
    ft = Sequential()
    ft.add(Dense(layers[0], input_dim=layers[0]))
    ft.add(GaussianNoise(0.5))

    for i in range(len(layers)-1):
        #print(i)
        ft.add(Dense(layers[i+1], activation = activation, weights = encoder[i], kernel_regularizer = regularizers.l1_l2(0.01)))
        
    for i in reversed(range(len(layers)-1)):
        #print(i)
        ft.add(Dense(layers[i], activation = activation, weights = decoder[i]))
    
    #ft.add(Dense(1, kernel_initializer = 'normal'))   
    ft.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    ft.fit(normX, normX, epochs = epoch, batch_size = batch, validation_data=(inputXt, inputXt))
    ft.summary()
    
    return ft

In [ ]:
tryla = fine_tuning(test, normX, normXt, trainY, testY, layers = layers, epoch = 1000, activation = 'relu', batch = 16, optimizer = 'adam', loss = 'mse')

In [ ]:
tryla.predict(x=normXt)

In [ ]:
tryla.save("fine_tune.hdf5")